In [1]:
class Homework(object):
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not(0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

In [2]:
galileo = Homework()
galileo.grade = 95

In [3]:
# property로 지정된 attribute들에 반복적으로 동일한 로직을 수행해야 함.
# 비효율적.
class Exam(object):
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
        
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value


In [5]:
# Grade를 descriptor로 구현한 모습
class Grade(object):
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
        self._value = value
        
# descriptor로 구현한 Grade를 사용할 Exam
class Exam(object):
    math_grade    = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    
# exam.write_grade = 40 is interpreted as:
# Exam.__dict__['writing_grade'].__set__(exam, 40)
# print(exam.write_grade) is interpreted as:
# print(Exam.__dict__['writing_grade'].__get__(exam, Exam))

In [7]:
# 위의 Grade 구현은 Exam 클래스 생성시에 만들어진 Grade 객체가 공유되기 때문에
# 아래와 같이 exam 클래스가 여러 instance인 경우 오동작할 수 있다.
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print 'Writing', first_exam.writing_grade
print 'Science', first_exam.science_grade

second_exam = Exam()
second_exam.writing_grade = 75
print 'Second', second_exam.writing_grade, 'is right'
print 'First', first_exam.writing_grade, 'is wrong'

Writing 82
Science 99
Second 75 is right
First 75 is wrong


In [12]:
from weakref import WeakKeyDictionary

# 그래서 instance마다 값을 따로 저장할 수 있도록 Grade를 구현해야 한다.
# Grade를 descriptor로 구현한 모습
class Grade(object):
    def __init__(self):
        # Exam instance가 모두 사라질 경우 이 dictionary에서도 사라져야 하므로 WeakKeyDictionary를 사용한다.
        # 그렇지 않으면 Exam instance가 한번 setting되면 사라지지 않고 계속 남게 됨.
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
        self._values[instance] = value
        
class Exam(object):
    math_grade    = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [14]:
# 위의 Grade 구현은 Exam 클래스 생성시에 만들어진 Grade 객체가 공유되기 때문에
# 아래와 같이 exam 클래스가 여러 instance인 경우 오동작할 수 있다.
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print 'Writing', first_exam.writing_grade
print 'Science', first_exam.science_grade

second_exam = Exam()
second_exam.writing_grade = 75
print 'Second', second_exam.writing_grade, 'is right'
print 'First', first_exam.writing_grade, 'is right'

Writing 82
Science 99
Second 75 is right
First 82 is right
